In [1]:
#importamos paquetes
import sys
sys.path.append('Src/')
from data_modificado import * #hay funciones que estan cambiadas en este script para adaptralas a nuestro dataset
from train_2 import * #este hubo que modificar una linea tambien
from transfer_learning import * #hubo que modificart lo mismo que en train_2
from test_functions import *
from layers import *
from utils import *
from loss import *
from metric import *
from results import *
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import itertools as it

#funciones
def read_df(
              metadata_names=['age','Temperature','Precipitation3Days'],
              random_state=42,
              otu_filename='../Datasets/otu_table_all_80.csv',
              metadata_filename='../Datasets/metadata_table_all_80.csv'):
    otu = pd.read_csv(otu_filename, index_col=0, header=None).T
    #print(otu.head())
    otu = otu.set_index('otuids')
    otu = otu.astype('int32')
    metadata = pd.read_csv(metadata_filename)
    #print(metadata.head())
    metadata = metadata.set_index('X.SampleID')
    metadata.head()
    domain = metadata[metadata_names]
    #if 'INBREDS' in metadata_names:
    #    domain = pd.concat([domain, pd.get_dummies(domain['INBREDS'], prefix='INBREDS')], axis=1)
    #    domain = domain.drop(['INBREDS'], axis=1)
    #elif 'Maize_Line' in metadata_names:
    #    domain = pd.concat([domain, pd.get_dummies(domain['Maize_Line'], prefix='Maize_Line')], axis=1)
    #    domain = domain.drop(['Maize_Line'], axis=1) 
    df = pd.concat([otu, domain], axis=1, sort=True, join='outer')
    #print(df.head())
    #data_microbioma = df[otu.columns].to_numpy(dtype=np.float32)
    #data_domain = df[domain.columns].to_numpy(dtype=np.float32)
    df_microbioma = df[otu.columns]
    df_domain = df[domain.columns]
    df_domain.head()
    df_microbioma_train, df_microbioma_no_train, df_domain_train, df_domain_no_train = \
        train_test_split(df_microbioma, df_domain, test_size=0.1, random_state=random_state)
    # Transfer learning subset
    df_microbioma_test, df_microbioma_transfer_learning, df_domain_test, df_domain_transfer_learning = \
        train_test_split(df_microbioma_no_train, df_domain_no_train, test_size=0.1, random_state=random_state)
    df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test = \
        train_test_split(df_microbioma_transfer_learning, df_domain_transfer_learning, test_size=0.3, random_state=random_state)
    
    return df_microbioma_train, df_microbioma_test, df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_train, df_domain_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test, otu.columns, domain.columns
    #return df_microbioma_train, df_microbioma_test, df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_train, df_domain_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test, otu.columns, domain.columns


def train_kfold_mod(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                batch_size, epochs, train_callbacks):
    all_models = model_fn()
    model, encoder_bioma, encoder_domain, decoder_bioma = all_models
    metrics_prefix = None
    if encoder_bioma is not None and encoder_domain is not None:
        x_train = (m_train, d_train)
        y_train = (m_train, m_train, z_train)
        x_test = (m_test, d_test)
        y_test = (m_test, m_test, z_test)
    elif encoder_bioma is not None:
        x_train = m_train
        y_train = m_train
        x_test = m_test
        y_test = m_test
        metrics_prefix = 'bioma'
    elif encoder_domain is not None:
        x_train = d_train
        y_train = m_train
        x_test = d_test
        y_test = m_test
        metrics_prefix = 'domain'

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(5000).batch(
        batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    r = model.fit(train_dataset,
                  epochs=epochs,
                  validation_data=val_dataset,
                  callbacks=train_callbacks,
                  verbose=0)
    if metrics_prefix is not None:
        old_keys = r.history
        r.history = {}
        for k, v in old_keys.items():
            if k == 'loss' or k == 'val_loss':
                new_key = k
            elif k.startswith('val_'):
                new_key = 'val_{}_{}'.format(metrics_prefix, k[4:])
            else:
                new_key = '{}_{}'.format(metrics_prefix, k)
            r.history[new_key] = v
    del val_dataset
    del train_dataset
    del x_train
    del y_train
    del x_test
    del y_test
    return r, all_models

def train_2(model_fn,
          data_microbioma,
          data_domain,
          latent_space=10,
          folds=5,
          epochs=20,
          batch_size=128,
          learning_rate_scheduler=ExpDecayScheluder(),
          random_seed=347,
          verbose=0):
    data_zeros_latent = np.zeros((data_microbioma.shape[0], latent_space), dtype=data_microbioma.dtype)
    results = []
    models = []
    train_callbacks = [
        callbacks.EarlyStopping(monitor='val_loss', patience=epochs + 1, restore_best_weights=True)]
    if verbose >= 0:
        train_callbacks += [TqdmCallback(verbose=verbose)]
    if learning_rate_scheduler is not None:
        train_callbacks += [learning_rate_scheduler.make()]

    if folds <= 1:
        m_train, m_test = data_microbioma, data_microbioma
        d_train, d_test = data_domain, data_domain
        z_train, z_test = data_zeros_latent, data_zeros_latent
        tf.random.set_seed(random_seed)
        r, m = train_kfold(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                           batch_size, epochs, train_callbacks)
        results.append(r)
        models.append(m)

    else: #EL PROBLEMA ESTA AQUI, QUE HACE FALTA UN 
        kf = KFold(n_splits=folds, random_state=random_seed, shuffle=True)
        tf.random.set_seed(random_seed)

        for train_index, test_index in kf.split(data_microbioma):
            m_train, m_test = data_microbioma[train_index], data_microbioma[test_index]
            #print(m_train)
            #d_train, d_test = data_domain[train_index], data_domain[test_index]
            if data_domain is None:
                d_train, d_test = None, None
            else:
                d_train, d_test = data_domain[train_index], data_domain[test_index]
            #print(d_train)
            #Esto de hacer el if else ha funcionado, pero no se si hace lo que debe bien
            z_train, z_test = data_zeros_latent[train_index], data_zeros_latent[test_index]
            r, m = train_kfold_mod(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                               batch_size, epochs, train_callbacks)
            results.append(r)
            models.append(m)
    return results, models

def perform_experiment_2_mod(cv_folds, epochs, batch_size, learning_rate, optimizer,
                       learning_rate_scheduler, input_transform, output_transform,
                       reconstruction_loss, latent_space, layers,
                       activation, activation_latent,
                       data_microbioma_train, data_domain_train,
                       show_results=True, device='/CPU:0'): #Show results cambiado de False  aTrue
    if input_transform is not None:
        input_transform = input_transform()
    #----------    
    if output_transform is not None:
        output_transform = output_transform()
    #----------      
    if reconstruction_loss.__class__.__name__ == 'MakeLoss':
        reconstruction_loss = reconstruction_loss.make()
    else:
        reconstruction_loss = reconstruction_loss()
    domain_layers = [l // 16 for l in layers] ####que es esto???? Esto es para las capas del domain
    #print(domain_layers)
    bioma_autoencoder = " -> ".join(["b"] +
                                    [str(l) for l in layers] +
                                    [str(latent_space)] +
                                    [str(l) for l in reversed(layers)] +
                                    ["b"])
    #---------- 
    #esto solo se utiliza para el texto, es irrelevante para nuestro error
    if data_domain_train is not None:
        domain_autoencoder = " -> ".join(["d"] +
                                     [str(l) for l in domain_layers] +
                                     [str(latent_space)] +
                                     [str(l) for l in reversed(layers)] +
                                     ["b"])
        
    else: 
        domain_autoencoder = " "
    #---------- 
    #donde se usa domain autoencoder?
    in_transform_name = input_transform.__class__.__name__ if input_transform else "none"
    out_transform_name = output_transform.__class__.__name__ if output_transform else "none"
    lr_scheduler_text = learning_rate_scheduler[
        1] if learning_rate_scheduler is not None else "none"
    lr_text = learning_rate if learning_rate_scheduler is not None else "constant = {}".format(
        learning_rate)
    learning_rate_scheduler = learning_rate_scheduler[
        0] if learning_rate_scheduler is not None else None
    optimizer = optimizer(learning_rate=learning_rate)
    #---------- 
    experiment_parameters = [
        ("Input transform", in_transform_name),
        ("Output transform", out_transform_name),
        ("Reconstruction Loss", reconstruction_loss.__class__.__name__),
        ("Latent Space", latent_space),
        ("Bioma Autoencoder", bioma_autoencoder),
        ("Domain Autoencoder", domain_autoencoder),
        ("Activation Encoder", activation),
        ("Activation Decoder", activation),
        ("Activation Latent", activation_latent),
        ("CV folds", cv_folds),
        ("Epochs", epochs),
        ("Batch Size", batch_size),
        ("Learning Rate Scheduler", lr_scheduler_text),
        ("Learning Rate", lr_text),
        ("Optimizer", optimizer.__class__.__name__),
    ]
    #----------  
    if show_results:
        md_text = ""
        md_text += "| Parameter             | Value         |\n"
        md_text += "|:----------------------|:--------------|\n"
        for n, v in experiment_parameters:
            md_text += "| {} | {} |\n".format(n, v)

        display(Markdown(md_text))
    #------------
    def create_model(print_data=False):
        bioma_shape=data_microbioma_train.shape[1]
        
        if data_domain_train is not None:
            domain_shape=data_domain_train.shape[1]
            #print("data_domain_train!=None")
        else:
            domain_shape=None
            #print("data_domain_train==None")
        models = autoencoder(bioma_shape=bioma_shape,
                             #bioma_shape=717,
                             domain_shape=domain_shape,
                             output_shape=bioma_shape,
                             #output_shape=717,
                             latent_space=latent_space,
                             bioma_layers=layers, #Esto es lo de [512,316]
                             domain_layers=domain_layers, #Esto son cada una de las layers divididas por 16
                             input_transform=input_transform,
                             output_transform=output_transform,
                             activation_function_encoder=activation,
                             activation_function_decoder=activation,
                             activation_function_latent=activation_latent)
        #Entiendo analizando lo demas que aqui NO esta el error
        #la funcion autoencoder esta en model.py (es la unica funcion en ese script)
        
        model, encoder_bioma, encoder_domain, decoder_bioma = models

        if print_data:
            plot_models(model, encoder_bioma, encoder_domain, decoder_bioma)
        compile_train(model,
                      encoder_bioma=encoder_bioma,
                      encoder_domain=encoder_domain,
                      reconstruction_error=reconstruction_loss,
                      encoded_comparison_error=losses.MeanAbsoluteError(),
                      metrics=get_experiment_metrics(input_transform, output_transform),
                      optimizer=optimizer)
        
        #print("He acabado create_model :)")
        return model, encoder_bioma, encoder_domain, decoder_bioma
    #-----------
    create_model(print_data=False)
    #-----------
    #Esta en esta seccion el problema, en train_2
    #print(data_domain_train)
    #print(latent_space)
    with tf.device(device):
        results, models = train_2(create_model,
                                data_microbioma_train,
                                data_domain_train,
                                latent_space=latent_space,
                                folds=cv_folds,
                                epochs=epochs,
                                batch_size=batch_size,
                                learning_rate_scheduler=learning_rate_scheduler,
                                verbose=-1)
    #----------
    validation_results = print_results(results, show_results=show_results)
    if show_results:
        display(Markdown("*************"))

    return experiment_parameters + validation_results, models, results

__params:__
 - "activat_func":["softmax","sigmoid","relu","tanh"]
 - "activ_ouput":["softmax","sigmoid","relu","tanh"]
 - "learning_rate":[0.01,0.001]

In [2]:
#Cargamos los datos
#nombres_metadatos = ["KCAL","PROT","TFAT","CARB","MOIS","ALC","CAFF","THEO","SUGR","FIBE","CALC","IRON","MAGN","PHOS","POTA","SODI","ZINC","COPP","SELE","VC","VB1","VB2","NIAC","VB6","FOLA","VB12","VARA","RET","BCAR","ACAR","CRYP","LYCO","LZ","ATOC","VK","CHOLE","SFAT","MFAT","PFAT","VITD","CHOLN"]
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='resultados_ana/datos_otus_metadatos_especies/otu_table_especies_80.csv',metadata_filename='datos-remoto/comidas/metadatos_comidas.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

#Preparamos las combinaciones pertinentes (5 mejores)
combinations = [[100,64,0.001,optimizers.Adam,15,[512,256],"tanh","tanh"],\
                [100,64,0.01,optimizers.Adam,15,[512,256],"tanh","tanh"],\
                [100,64,0.001,optimizers.Adam,15,[512,256,128],"tanh","tanh"],\
                [100,96,0.001,optimizers.Adam,15,[512,256],"tanh","tanh"],\
                [100,64,0.001,optimizers.Adam,15,[512,256,128,64],"tanh","tanh"]]

### 0.001, tanh, tanh

In [4]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation=None)(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4720536768436432 | 0.4720536768436432 | 0.4720536768436432 |
| pearson_corr | 0.6329877972602844 | 0.6329877972602844 | 0.6329877972602844 |
| jensen_shannon_divergence | 1.2707825899124146 | 1.2707825899124146 | 1.2707825899124146 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4834246039390564 | 0.4834246039390564 | 0.4834246039390564 |
| pearson_corr | 0.5842662453651428 | 0.5842662453651428 | 0.5842662453651428 |
| jensen_shannon_divergence | 1.3304566144943237 | 1.3304566144943237 | 1.3304566144943237 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46799615025520325 | 0.46799615025520325 | 0.46799615025520325 |
| pearson_corr | 0.6263048648834229 | 0.6263048648834229 | 0.6263048648834229 |
| jensen_shannon_divergence | 1.2731940746307373 | 1.2731940746307373 | 1.2731940746307373 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4979006052017212 | 0.4979006052017212 | 0.4979006052017212 |
| pearson_corr | 0.5871923565864563 | 0.5871923565864563 | 0.5871923565864563 |
| jensen_shannon_divergence | 1.4102447032928467 | 1.4102447032928467 | 1.4102447032928467 |


----------------------------



### 0.001, tanh, softmax

In [5]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7997022867202759 | 0.7997022867202759 | 0.7997022867202759 |
| pearson_corr | 0.22693565487861633 | 0.22693565487861633 | 0.22693565487861633 |
| jensen_shannon_divergence | 3.305359125137329 | 3.305359125137329 | 3.305359125137329 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5316540598869324 | 0.5316540598869324 | 0.5316540598869324 |
| pearson_corr | 0.5374889969825745 | 0.5374889969825745 | 0.5374889969825745 |
| jensen_shannon_divergence | 1.4253900051116943 | 1.4253900051116943 | 1.4253900051116943 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7775302529335022 | 0.7775302529335022 | 0.7775302529335022 |
| pearson_corr | 0.3708404302597046 | 0.3708404302597046 | 0.3708404302597046 |
| jensen_shannon_divergence | 3.0087342262268066 | 3.0087342262268066 | 3.0087342262268066 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8028163313865662 | 0.8028163313865662 | 0.8028163313865662 |
| pearson_corr | 0.08904334157705307 | 0.08904334157705307 | 0.08904334157705307 |
| jensen_shannon_divergence | 3.853548765182495 | 3.853548765182495 | 3.853548765182495 |


----------------------------



### 0.001, tanh, sigmoid

In [6]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8835567831993103 | 0.8835567831993103 | 0.8835567831993103 |
| pearson_corr | 0.026119906455278397 | 0.026119906455278397 | 0.026119906455278397 |
| jensen_shannon_divergence | 5.832303524017334 | 5.832303524017334 | 5.832303524017334 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.542927622795105 | 0.542927622795105 | 0.542927622795105 |
| pearson_corr | 0.5165515542030334 | 0.5165515542030334 | 0.5165515542030334 |
| jensen_shannon_divergence | 1.4887462854385376 | 1.4887462854385376 | 1.4887462854385376 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8696870803833008 | 0.8696870803833008 | 0.8696870803833008 |
| pearson_corr | 0.03776601701974869 | 0.03776601701974869 | 0.03776601701974869 |
| jensen_shannon_divergence | 5.033145427703857 | 5.033145427703857 | 5.033145427703857 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7882190346717834 | 0.7882190346717834 | 0.7882190346717834 |
| pearson_corr | 0.0959259495139122 | 0.0959259495139122 | 0.0959259495139122 |
| jensen_shannon_divergence | 3.6205620765686035 | 3.6205620765686035 | 3.6205620765686035 |


----------------------------



### 0.001, tanh, relu

In [7]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8384332060813904 | 0.8384332060813904 | 0.8384332060813904 |
| pearson_corr | 0.04139041528105736 | 0.04139041528105736 | 0.04139041528105736 |
| jensen_shannon_divergence | 4.300887584686279 | 4.300887584686279 | 4.300887584686279 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5694420337677002 | 0.5694420337677002 | 0.5694420337677002 |
| pearson_corr | 0.45317238569259644 | 0.45317238569259644 | 0.45317238569259644 |
| jensen_shannon_divergence | 1.6370866298675537 | 1.6370866298675537 | 1.6370866298675537 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6702282428741455 | 0.6702282428741455 | 0.6702282428741455 |
| pearson_corr | 0.401256799697876 | 0.401256799697876 | 0.401256799697876 |
| jensen_shannon_divergence | 2.368807315826416 | 2.368807315826416 | 2.368807315826416 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7334422469139099 | 0.7334422469139099 | 0.7334422469139099 |
| pearson_corr | 0.1713225245475769 | 0.1713225245475769 | 0.1713225245475769 |
| jensen_shannon_divergence | 2.9648613929748535 | 2.9648613929748535 | 2.9648613929748535 |


----------------------------



### 0.001, softmax, tanh

In [8]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
  
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47427263855934143 | 0.47427263855934143 | 0.47427263855934143 |
| pearson_corr | 0.6472010612487793 | 0.6472010612487793 | 0.6472010612487793 |
| jensen_shannon_divergence | 1.187390923500061 | 1.187390923500061 | 1.187390923500061 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.483003705739975 | 0.483003705739975 | 0.483003705739975 |
| pearson_corr | 0.6159290671348572 | 0.6159290671348572 | 0.6159290671348572 |
| jensen_shannon_divergence | 1.3051222562789917 | 1.3051222562789917 | 1.3051222562789917 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.48590561747550964 | 0.48590561747550964 | 0.48590561747550964 |
| pearson_corr | 0.6359436511993408 | 0.6359436511993408 | 0.6359436511993408 |
| jensen_shannon_divergence | 1.2253504991531372 | 1.2253504991531372 | 1.2253504991531372 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4675416350364685 | 0.4675416350364685 | 0.4675416350364685 |
| pearson_corr | 0.6479435563087463 | 0.6479435563087463 | 0.6479435563087463 |
| jensen_shannon_divergence | 1.1873410940170288 | 1.1873410940170288 | 1.1873410940170288 |


----------------------------



### 0.001, softmax, softmax

In [9]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8003240823745728 | 0.8003240823745728 | 0.8003240823745728 |
| pearson_corr | 0.22484645247459412 | 0.22484645247459412 | 0.22484645247459412 |
| jensen_shannon_divergence | 3.313708782196045 | 3.313708782196045 | 3.313708782196045 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5344032645225525 | 0.5344032645225525 | 0.5344032645225525 |
| pearson_corr | 0.5328337550163269 | 0.5328337550163269 | 0.5328337550163269 |
| jensen_shannon_divergence | 1.4346024990081787 | 1.4346024990081787 | 1.4346024990081787 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7775967121124268 | 0.7775967121124268 | 0.7775967121124268 |
| pearson_corr | 0.3731094002723694 | 0.3731094002723694 | 0.3731094002723694 |
| jensen_shannon_divergence | 3.009721517562866 | 3.009721517562866 | 3.009721517562866 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.792597234249115 | 0.792597234249115 | 0.792597234249115 |
| pearson_corr | 0.09286366403102875 | 0.09286366403102875 | 0.09286366403102875 |
| jensen_shannon_divergence | 3.6986167430877686 | 3.6986167430877686 | 3.6986167430877686 |


----------------------------



### 0.001, softmax, sigmoid

In [10]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8913232684135437 | 0.8913232684135437 | 0.8913232684135437 |
| pearson_corr | 0.024156754836440086 | 0.024156754836440086 | 0.024156754836440086 |
| jensen_shannon_divergence | 6.07019567489624 | 6.07019567489624 | 6.07019567489624 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.540776252746582 | 0.540776252746582 | 0.540776252746582 |
| pearson_corr | 0.5133913159370422 | 0.5133913159370422 | 0.5133913159370422 |
| jensen_shannon_divergence | 1.4387857913970947 | 1.4387857913970947 | 1.4387857913970947 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8906101584434509 | 0.8906101584434509 | 0.8906101584434509 |
| pearson_corr | 0.03113805688917637 | 0.03113805688917637 | 0.03113805688917637 |
| jensen_shannon_divergence | 5.3682990074157715 | 5.3682990074157715 | 5.3682990074157715 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7961539626121521 | 0.7961539626121521 | 0.7961539626121521 |
| pearson_corr | 0.09182258695363998 | 0.09182258695363998 | 0.09182258695363998 |
| jensen_shannon_divergence | 3.787247657775879 | 3.787247657775879 | 3.787247657775879 |


----------------------------



### 0.001, softmax, relu

In [11]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')


    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8809365034103394 | 0.8809365034103394 | 0.8809365034103394 |
| pearson_corr | 0.04086493328213692 | 0.04086493328213692 | 0.04086493328213692 |
| jensen_shannon_divergence | 5.446573734283447 | 5.446573734283447 | 5.446573734283447 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.542633056640625 | 0.542633056640625 | 0.542633056640625 |
| pearson_corr | 0.517400324344635 | 0.517400324344635 | 0.517400324344635 |
| jensen_shannon_divergence | 1.4949851036071777 | 1.4949851036071777 | 1.4949851036071777 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7204163074493408 | 0.7204163074493408 | 0.7204163074493408 |
| pearson_corr | 0.28769007325172424 | 0.28769007325172424 | 0.28769007325172424 |
| jensen_shannon_divergence | 3.077767848968506 | 3.077767848968506 | 3.077767848968506 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7635713219642639 | 0.7635713219642639 | 0.7635713219642639 |
| pearson_corr | 0.11981847137212753 | 0.11981847137212753 | 0.11981847137212753 |
| jensen_shannon_divergence | 3.217271089553833 | 3.217271089553833 | 3.217271089553833 |


----------------------------



### 0.001, sigmoid, tanh

In [12]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.457141637802124 | 0.457141637802124 | 0.457141637802124 |
| pearson_corr | 0.6803795099258423 | 0.6803795099258423 | 0.6803795099258423 |
| jensen_shannon_divergence | 1.1075595617294312 | 1.1075595617294312 | 1.1075595617294312 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4610236585140228 | 0.4610236585140228 | 0.4610236585140228 |
| pearson_corr | 0.6397521495819092 | 0.6397521495819092 | 0.6397521495819092 |
| jensen_shannon_divergence | 1.2274417877197266 | 1.2274417877197266 | 1.2274417877197266 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47365236282348633 | 0.47365236282348633 | 0.47365236282348633 |
| pearson_corr | 0.6540795564651489 | 0.6540795564651489 | 0.6540795564651489 |
| jensen_shannon_divergence | 1.1860451698303223 | 1.1860451698303223 | 1.1860451698303223 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4547685980796814 | 0.4547685980796814 | 0.4547685980796814 |
| pearson_corr | 0.6678120493888855 | 0.6678120493888855 | 0.6678120493888855 |
| jensen_shannon_divergence | 1.1508808135986328 | 1.1508808135986328 | 1.1508808135986328 |


----------------------------



### 0.001, sigmoid, softmax

In [13]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7995879054069519 | 0.7995879054069519 | 0.7995879054069519 |
| pearson_corr | 0.22784750163555145 | 0.22784750163555145 | 0.22784750163555145 |
| jensen_shannon_divergence | 3.3038275241851807 | 3.3038275241851807 | 3.3038275241851807 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5341218709945679 | 0.5341218709945679 | 0.5341218709945679 |
| pearson_corr | 0.5333413481712341 | 0.5333413481712341 | 0.5333413481712341 |
| jensen_shannon_divergence | 1.4358595609664917 | 1.4358595609664917 | 1.4358595609664917 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.777350902557373 | 0.777350902557373 | 0.777350902557373 |
| pearson_corr | 0.37099677324295044 | 0.37099677324295044 | 0.37099677324295044 |
| jensen_shannon_divergence | 3.006699800491333 | 3.006699800491333 | 3.006699800491333 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7968201041221619 | 0.7968201041221619 | 0.7968201041221619 |
| pearson_corr | 0.09128665924072266 | 0.09128665924072266 | 0.09128665924072266 |
| jensen_shannon_divergence | 3.785855293273926 | 3.785855293273926 | 3.785855293273926 |


----------------------------



### 0.001, sigmoid, sigmoid

In [14]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8842914700508118 | 0.8842914700508118 | 0.8842914700508118 |
| pearson_corr | 0.025958992540836334 | 0.025958992540836334 | 0.025958992540836334 |
| jensen_shannon_divergence | 5.852493762969971 | 5.852493762969971 | 5.852493762969971 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.542815089225769 | 0.542815089225769 | 0.542815089225769 |
| pearson_corr | 0.514980673789978 | 0.514980673789978 | 0.514980673789978 |
| jensen_shannon_divergence | 1.4853767156600952 | 1.4853767156600952 | 1.4853767156600952 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8704866170883179 | 0.8704866170883179 | 0.8704866170883179 |
| pearson_corr | 0.03753620386123657 | 0.03753620386123657 | 0.03753620386123657 |
| jensen_shannon_divergence | 5.045878887176514 | 5.045878887176514 | 5.045878887176514 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7885849475860596 | 0.7885849475860596 | 0.7885849475860596 |
| pearson_corr | 0.09629328548908234 | 0.09629328548908234 | 0.09629328548908234 |
| jensen_shannon_divergence | 3.6265220642089844 | 3.6265220642089844 | 3.6265220642089844 |


----------------------------



### 0.001, sigmoid, relu

In [15]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8358638286590576 | 0.8358638286590576 | 0.8358638286590576 |
| pearson_corr | 0.043978288769721985 | 0.043978288769721985 | 0.043978288769721985 |
| jensen_shannon_divergence | 4.352478981018066 | 4.352478981018066 | 4.352478981018066 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5686659812927246 | 0.5686659812927246 | 0.5686659812927246 |
| pearson_corr | 0.44998839497566223 | 0.44998839497566223 | 0.44998839497566223 |
| jensen_shannon_divergence | 1.630423665046692 | 1.630423665046692 | 1.630423665046692 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7160576581954956 | 0.7160576581954956 | 0.7160576581954956 |
| pearson_corr | 0.3827992379665375 | 0.3827992379665375 | 0.3827992379665375 |
| jensen_shannon_divergence | 2.6868503093719482 | 2.6868503093719482 | 2.6868503093719482 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7694165110588074 | 0.7694165110588074 | 0.7694165110588074 |
| pearson_corr | 0.11212030053138733 | 0.11212030053138733 | 0.11212030053138733 |
| jensen_shannon_divergence | 3.2860405445098877 | 3.2860405445098877 | 3.2860405445098877 |


----------------------------



### 0.001, relu, tanh

In [16]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4667639136314392 | 0.4667639136314392 | 0.4667639136314392 |
| pearson_corr | 0.6499099135398865 | 0.6499099135398865 | 0.6499099135398865 |
| jensen_shannon_divergence | 1.2084590196609497 | 1.2084590196609497 | 1.2084590196609497 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4823876917362213 | 0.4823876917362213 | 0.4823876917362213 |
| pearson_corr | 0.6094485521316528 | 0.6094485521316528 | 0.6094485521316528 |
| jensen_shannon_divergence | 1.3066725730895996 | 1.3066725730895996 | 1.3066725730895996 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46330079436302185 | 0.46330079436302185 | 0.46330079436302185 |
| pearson_corr | 0.6491295695304871 | 0.6491295695304871 | 0.6491295695304871 |
| jensen_shannon_divergence | 1.244844675064087 | 1.244844675064087 | 1.244844675064087 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47807538509368896 | 0.47807538509368896 | 0.47807538509368896 |
| pearson_corr | 0.6351757645606995 | 0.6351757645606995 | 0.6351757645606995 |
| jensen_shannon_divergence | 1.2785111665725708 | 1.2785111665725708 | 1.2785111665725708 |


----------------------------



### 0.001, relu, softmax

In [17]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.799610435962677 | 0.799610435962677 | 0.799610435962677 |
| pearson_corr | 0.22604671120643616 | 0.22604671120643616 | 0.22604671120643616 |
| jensen_shannon_divergence | 3.304995059967041 | 3.304995059967041 | 3.304995059967041 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5342480540275574 | 0.5342480540275574 | 0.5342480540275574 |
| pearson_corr | 0.5338294506072998 | 0.5338294506072998 | 0.5338294506072998 |
| jensen_shannon_divergence | 1.4401441812515259 | 1.4401441812515259 | 1.4401441812515259 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7770203948020935 | 0.7770203948020935 | 0.7770203948020935 |
| pearson_corr | 0.37251922488212585 | 0.37251922488212585 | 0.37251922488212585 |
| jensen_shannon_divergence | 3.0037782192230225 | 3.0037782192230225 | 3.0037782192230225 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8024715781211853 | 0.8024715781211853 | 0.8024715781211853 |
| pearson_corr | 0.08498515188694 | 0.08498515188694 | 0.08498515188694 |
| jensen_shannon_divergence | 3.87284779548645 | 3.87284779548645 | 3.87284779548645 |


----------------------------



### 0.001, relu, sigmoid

In [18]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8447081446647644 | 0.8447081446647644 | 0.8447081446647644 |
| pearson_corr | 0.03996548801660538 | 0.03996548801660538 | 0.03996548801660538 |
| jensen_shannon_divergence | 4.466740608215332 | 4.466740608215332 | 4.466740608215332 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5450881719589233 | 0.5450881719589233 | 0.5450881719589233 |
| pearson_corr | 0.5081996321678162 | 0.5081996321678162 | 0.5081996321678162 |
| jensen_shannon_divergence | 1.5090771913528442 | 1.5090771913528442 | 1.5090771913528442 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6502818465232849 | 0.6502818465232849 | 0.6502818465232849 |
| pearson_corr | 0.4411470592021942 | 0.4411470592021942 | 0.4411470592021942 |
| jensen_shannon_divergence | 2.2265570163726807 | 2.2265570163726807 | 2.2265570163726807 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7970131635665894 | 0.7970131635665894 | 0.7970131635665894 |
| pearson_corr | 0.09804282337427139 | 0.09804282337427139 | 0.09804282337427139 |
| jensen_shannon_divergence | 3.6617798805236816 | 3.6617798805236816 | 3.6617798805236816 |


----------------------------



### 0.001, relu, relu

In [19]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.751838743686676 | 0.751838743686676 | 0.751838743686676 |
| pearson_corr | 0.17121857404708862 | 0.17121857404708862 | 0.17121857404708862 |
| jensen_shannon_divergence | 3.2655718326568604 | 3.2655718326568604 | 3.2655718326568604 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5465261340141296 | 0.5465261340141296 | 0.5465261340141296 |
| pearson_corr | 0.5031484365463257 | 0.5031484365463257 | 0.5031484365463257 |
| jensen_shannon_divergence | 1.5126539468765259 | 1.5126539468765259 | 1.5126539468765259 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.663550615310669 | 0.663550615310669 | 0.663550615310669 |
| pearson_corr | 0.40596213936805725 | 0.40596213936805725 | 0.40596213936805725 |
| jensen_shannon_divergence | 2.311481237411499 | 2.311481237411499 | 2.311481237411499 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7885576486587524 | 0.7885576486587524 | 0.7885576486587524 |
| pearson_corr | 0.09605186432600021 | 0.09605186432600021 | 0.09605186432600021 |
| jensen_shannon_divergence | 3.60886812210083 | 3.60886812210083 | 3.60886812210083 |


----------------------------



---------------------------
### 0.01, tanh, tanh

In [20]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4741678535938263 | 0.4741678535938263 | 0.4741678535938263 |
| pearson_corr | 0.6346926689147949 | 0.6346926689147949 | 0.6346926689147949 |
| jensen_shannon_divergence | 1.267013430595398 | 1.267013430595398 | 1.267013430595398 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47666338086128235 | 0.47666338086128235 | 0.47666338086128235 |
| pearson_corr | 0.6153964400291443 | 0.6153964400291443 | 0.6153964400291443 |
| jensen_shannon_divergence | 1.3824340105056763 | 1.3824340105056763 | 1.3824340105056763 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5055676102638245 | 0.5055676102638245 | 0.5055676102638245 |
| pearson_corr | 0.5774533152580261 | 0.5774533152580261 | 0.5774533152580261 |
| jensen_shannon_divergence | 1.372916579246521 | 1.372916579246521 | 1.372916579246521 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5012011528015137 | 0.5012011528015137 | 0.5012011528015137 |
| pearson_corr | 0.6110777258872986 | 0.6110777258872986 | 0.6110777258872986 |
| jensen_shannon_divergence | 1.3201346397399902 | 1.3201346397399902 | 1.3201346397399902 |


----------------------------



### 0.01, tanh, softmax

In [21]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8005775213241577 | 0.8005775213241577 | 0.8005775213241577 |
| pearson_corr | 0.21870818734169006 | 0.21870818734169006 | 0.21870818734169006 |
| jensen_shannon_divergence | 3.316744565963745 | 3.316744565963745 | 3.316744565963745 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.532950758934021 | 0.532950758934021 | 0.532950758934021 |
| pearson_corr | 0.5347887873649597 | 0.5347887873649597 | 0.5347887873649597 |
| jensen_shannon_divergence | 1.4293347597122192 | 1.4293347597122192 | 1.4293347597122192 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7764288187026978 | 0.7764288187026978 | 0.7764288187026978 |
| pearson_corr | 0.37425097823143005 | 0.37425097823143005 | 0.37425097823143005 |
| jensen_shannon_divergence | 2.99788498878479 | 2.99788498878479 | 2.99788498878479 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.805120587348938 | 0.805120587348938 | 0.805120587348938 |
| pearson_corr | 0.08161229640245438 | 0.08161229640245438 | 0.08161229640245438 |
| jensen_shannon_divergence | 3.910930633544922 | 3.910930633544922 | 3.910930633544922 |


----------------------------



### 0.01, tanh, sigmoid

In [22]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.882674515247345 | 0.882674515247345 | 0.882674515247345 |
| pearson_corr | 0.026525329798460007 | 0.026525329798460007 | 0.026525329798460007 |
| jensen_shannon_divergence | 5.781524181365967 | 5.781524181365967 | 5.781524181365967 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5366344451904297 | 0.5366344451904297 | 0.5366344451904297 |
| pearson_corr | 0.5250107049942017 | 0.5250107049942017 | 0.5250107049942017 |
| jensen_shannon_divergence | 1.4622832536697388 | 1.4622832536697388 | 1.4622832536697388 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8544814586639404 | 0.8544814586639404 | 0.8544814586639404 |
| pearson_corr | 0.041717007756233215 | 0.041717007756233215 | 0.041717007756233215 |
| jensen_shannon_divergence | 4.787315845489502 | 4.787315845489502 | 4.787315845489502 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8011597394943237 | 0.8011597394943237 | 0.8011597394943237 |
| pearson_corr | 0.08368188142776489 | 0.08368188142776489 | 0.08368188142776489 |
| jensen_shannon_divergence | 3.839181661605835 | 3.839181661605835 | 3.839181661605835 |


----------------------------



### 0.01, tanh, relu

In [23]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8456486463546753 | 0.8456486463546753 | 0.8456486463546753 |
| pearson_corr | 0.03881436586380005 | 0.03881436586380005 | 0.03881436586380005 |
| jensen_shannon_divergence | 4.4092559814453125 | 4.4092559814453125 | 4.4092559814453125 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5650495290756226 | 0.5650495290756226 | 0.5650495290756226 |
| pearson_corr | 0.47952720522880554 | 0.47952720522880554 | 0.47952720522880554 |
| jensen_shannon_divergence | 1.605000376701355 | 1.605000376701355 | 1.605000376701355 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6678604483604431 | 0.6678604483604431 | 0.6678604483604431 |
| pearson_corr | 0.40306317806243896 | 0.40306317806243896 | 0.40306317806243896 |
| jensen_shannon_divergence | 2.3767216205596924 | 2.3767216205596924 | 2.3767216205596924 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8828073143959045 | 0.8828073143959045 | 0.8828073143959045 |
| pearson_corr | 0.009501475840806961 | 0.009501475840806961 | 0.009501475840806961 |
| jensen_shannon_divergence | 5.410399913787842 | 5.410399913787842 | 5.410399913787842 |


----------------------------



### 0.01, softmax, tanh

In [24]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46952682733535767 | 0.46952682733535767 | 0.46952682733535767 |
| pearson_corr | 0.6614938378334045 | 0.6614938378334045 | 0.6614938378334045 |
| jensen_shannon_divergence | 1.154409646987915 | 1.154409646987915 | 1.154409646987915 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4890241324901581 | 0.4890241324901581 | 0.4890241324901581 |
| pearson_corr | 0.6035942435264587 | 0.6035942435264587 | 0.6035942435264587 |
| jensen_shannon_divergence | 1.3305336236953735 | 1.3305336236953735 | 1.3305336236953735 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.48047780990600586 | 0.48047780990600586 | 0.48047780990600586 |
| pearson_corr | 0.6507522463798523 | 0.6507522463798523 | 0.6507522463798523 |
| jensen_shannon_divergence | 1.20634925365448 | 1.20634925365448 | 1.20634925365448 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4759860634803772 | 0.4759860634803772 | 0.4759860634803772 |
| pearson_corr | 0.6354849934577942 | 0.6354849934577942 | 0.6354849934577942 |
| jensen_shannon_divergence | 1.2318958044052124 | 1.2318958044052124 | 1.2318958044052124 |


----------------------------



### 0.01, softmax, softmax

In [25]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7998688817024231 | 0.7998688817024231 | 0.7998688817024231 |
| pearson_corr | 0.2255842387676239 | 0.2255842387676239 | 0.2255842387676239 |
| jensen_shannon_divergence | 3.307570219039917 | 3.307570219039917 | 3.307570219039917 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5339785218238831 | 0.5339785218238831 | 0.5339785218238831 |
| pearson_corr | 0.5336011052131653 | 0.5336011052131653 | 0.5336011052131653 |
| jensen_shannon_divergence | 1.4353115558624268 | 1.4353115558624268 | 1.4353115558624268 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7776256203651428 | 0.7776256203651428 | 0.7776256203651428 |
| pearson_corr | 0.3704560101032257 | 0.3704560101032257 | 0.3704560101032257 |
| jensen_shannon_divergence | 3.0099265575408936 | 3.0099265575408936 | 3.0099265575408936 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7959318161010742 | 0.7959318161010742 | 0.7959318161010742 |
| pearson_corr | 0.08929038792848587 | 0.08929038792848587 | 0.08929038792848587 |
| jensen_shannon_divergence | 3.751464605331421 | 3.751464605331421 | 3.751464605331421 |


----------------------------



### 0.01, softmax, sigmoid

In [26]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8837308883666992 | 0.8837308883666992 | 0.8837308883666992 |
| pearson_corr | 0.026023488491773605 | 0.026023488491773605 | 0.026023488491773605 |
| jensen_shannon_divergence | 5.8306450843811035 | 5.8306450843811035 | 5.8306450843811035 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5425686240196228 | 0.5425686240196228 | 0.5425686240196228 |
| pearson_corr | 0.5169673562049866 | 0.5169673562049866 | 0.5169673562049866 |
| jensen_shannon_divergence | 1.4896037578582764 | 1.4896037578582764 | 1.4896037578582764 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8696843385696411 | 0.8696843385696411 | 0.8696843385696411 |
| pearson_corr | 0.03786506503820419 | 0.03786506503820419 | 0.03786506503820419 |
| jensen_shannon_divergence | 5.030529022216797 | 5.030529022216797 | 5.030529022216797 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.789222240447998 | 0.789222240447998 | 0.789222240447998 |
| pearson_corr | 0.09478697180747986 | 0.09478697180747986 | 0.09478697180747986 |
| jensen_shannon_divergence | 3.6353600025177 | 3.6353600025177 | 3.6353600025177 |


----------------------------



### 0.01, softmax, relu

In [27]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8794893026351929 | 0.8794893026351929 | 0.8794893026351929 |
| pearson_corr | 0.04102536663413048 | 0.04102536663413048 | 0.04102536663413048 |
| jensen_shannon_divergence | 5.392488956451416 | 5.392488956451416 | 5.392488956451416 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5433120727539062 | 0.5433120727539062 | 0.5433120727539062 |
| pearson_corr | 0.5152749419212341 | 0.5152749419212341 | 0.5152749419212341 |
| jensen_shannon_divergence | 1.4939675331115723 | 1.4939675331115723 | 1.4939675331115723 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7200010418891907 | 0.7200010418891907 | 0.7200010418891907 |
| pearson_corr | 0.2876594364643097 | 0.2876594364643097 | 0.2876594364643097 |
| jensen_shannon_divergence | 3.0607237815856934 | 3.0607237815856934 | 3.0607237815856934 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7672832012176514 | 0.7672832012176514 | 0.7672832012176514 |
| pearson_corr | 0.11482517421245575 | 0.11482517421245575 | 0.11482517421245575 |
| jensen_shannon_divergence | 3.270242214202881 | 3.270242214202881 | 3.270242214202881 |


----------------------------



### 0.01, sigmoid, tanh

In [28]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.43542131781578064 | 0.43542131781578064 | 0.43542131781578064 |
| pearson_corr | 0.6711931824684143 | 0.6711931824684143 | 0.6711931824684143 |
| jensen_shannon_divergence | 1.128952145576477 | 1.128952145576477 | 1.128952145576477 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4708465039730072 | 0.4708465039730072 | 0.4708465039730072 |
| pearson_corr | 0.6156988143920898 | 0.6156988143920898 | 0.6156988143920898 |
| jensen_shannon_divergence | 1.2696269750595093 | 1.2696269750595093 | 1.2696269750595093 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4420377016067505 | 0.4420377016067505 | 0.4420377016067505 |
| pearson_corr | 0.6694134473800659 | 0.6694134473800659 | 0.6694134473800659 |
| jensen_shannon_divergence | 1.1252961158752441 | 1.1252961158752441 | 1.1252961158752441 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4699990153312683 | 0.4699990153312683 | 0.4699990153312683 |
| pearson_corr | 0.6520907282829285 | 0.6520907282829285 | 0.6520907282829285 |
| jensen_shannon_divergence | 1.2044124603271484 | 1.2044124603271484 | 1.2044124603271484 |


----------------------------



### 0.01, sigmoid, softmax

In [29]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8012933135032654 | 0.8012933135032654 | 0.8012933135032654 |
| pearson_corr | 0.2146243005990982 | 0.2146243005990982 | 0.2146243005990982 |
| jensen_shannon_divergence | 3.32645583152771 | 3.32645583152771 | 3.32645583152771 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.534538745880127 | 0.534538745880127 | 0.534538745880127 |
| pearson_corr | 0.5302342772483826 | 0.5302342772483826 | 0.5302342772483826 |
| jensen_shannon_divergence | 1.4349167346954346 | 1.4349167346954346 | 1.4349167346954346 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7785159945487976 | 0.7785159945487976 | 0.7785159945487976 |
| pearson_corr | 0.3662720024585724 | 0.3662720024585724 | 0.3662720024585724 |
| jensen_shannon_divergence | 3.020679235458374 | 3.020679235458374 | 3.020679235458374 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8014028072357178 | 0.8014028072357178 | 0.8014028072357178 |
| pearson_corr | 0.08501008152961731 | 0.08501008152961731 | 0.08501008152961731 |
| jensen_shannon_divergence | 3.839092254638672 | 3.839092254638672 | 3.839092254638672 |


----------------------------



### 0.01, sigmoid, sigmoid

In [30]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8803465962409973 | 0.8803465962409973 | 0.8803465962409973 |
| pearson_corr | 0.02682524360716343 | 0.02682524360716343 | 0.02682524360716343 |
| jensen_shannon_divergence | 5.765804290771484 | 5.765804290771484 | 5.765804290771484 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5407482981681824 | 0.5407482981681824 | 0.5407482981681824 |
| pearson_corr | 0.5207853317260742 | 0.5207853317260742 | 0.5207853317260742 |
| jensen_shannon_divergence | 1.4848875999450684 | 1.4848875999450684 | 1.4848875999450684 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.862305760383606 | 0.862305760383606 | 0.862305760383606 |
| pearson_corr | 0.03962424024939537 | 0.03962424024939537 | 0.03962424024939537 |
| jensen_shannon_divergence | 4.921187400817871 | 4.921187400817871 | 4.921187400817871 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.798485279083252 | 0.798485279083252 | 0.798485279083252 |
| pearson_corr | 0.08610024303197861 | 0.08610024303197861 | 0.08610024303197861 |
| jensen_shannon_divergence | 3.792341470718384 | 3.792341470718384 | 3.792341470718384 |


----------------------------



### 0.01, sigmoid, relu

In [31]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.836516261100769 | 0.836516261100769 | 0.836516261100769 |
| pearson_corr | 0.04364596679806709 | 0.04364596679806709 | 0.04364596679806709 |
| jensen_shannon_divergence | 4.360780239105225 | 4.360780239105225 | 4.360780239105225 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.566038191318512 | 0.566038191318512 | 0.566038191318512 |
| pearson_corr | 0.4770541787147522 | 0.4770541787147522 | 0.4770541787147522 |
| jensen_shannon_divergence | 1.6016992330551147 | 1.6016992330551147 | 1.6016992330551147 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7151888012886047 | 0.7151888012886047 | 0.7151888012886047 |
| pearson_corr | 0.38348647952079773 | 0.38348647952079773 | 0.38348647952079773 |
| jensen_shannon_divergence | 2.6843955516815186 | 2.6843955516815186 | 2.6843955516815186 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7117684483528137 | 0.7117684483528137 | 0.7117684483528137 |
| pearson_corr | 0.3145443797111511 | 0.3145443797111511 | 0.3145443797111511 |
| jensen_shannon_divergence | 2.50319504737854 | 2.50319504737854 | 2.50319504737854 |


----------------------------



### 0.01, relu, tanh

In [32]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4735003411769867 | 0.4735003411769867 | 0.4735003411769867 |
| pearson_corr | 0.6262132525444031 | 0.6262132525444031 | 0.6262132525444031 |
| jensen_shannon_divergence | 1.2198717594146729 | 1.2198717594146729 | 1.2198717594146729 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4837448000907898 | 0.4837448000907898 | 0.4837448000907898 |
| pearson_corr | 0.6186399459838867 | 0.6186399459838867 | 0.6186399459838867 |
| jensen_shannon_divergence | 1.3062366247177124 | 1.3062366247177124 | 1.3062366247177124 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47182348370552063 | 0.47182348370552063 | 0.47182348370552063 |
| pearson_corr | 0.6103520393371582 | 0.6103520393371582 | 0.6103520393371582 |
| jensen_shannon_divergence | 1.2433093786239624 | 1.2433093786239624 | 1.2433093786239624 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.470007061958313 | 0.470007061958313 | 0.470007061958313 |
| pearson_corr | 0.6438169479370117 | 0.6438169479370117 | 0.6438169479370117 |
| jensen_shannon_divergence | 1.1997673511505127 | 1.1997673511505127 | 1.1997673511505127 |


----------------------------



### 0.01, relu, softmax

In [33]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7998332381248474 | 0.7998332381248474 | 0.7998332381248474 |
| pearson_corr | 0.22476382553577423 | 0.22476382553577423 | 0.22476382553577423 |
| jensen_shannon_divergence | 3.308997869491577 | 3.308997869491577 | 3.308997869491577 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5339768528938293 | 0.5339768528938293 | 0.5339768528938293 |
| pearson_corr | 0.5311855673789978 | 0.5311855673789978 | 0.5311855673789978 |
| jensen_shannon_divergence | 1.4322865009307861 | 1.4322865009307861 | 1.4322865009307861 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7773314714431763 | 0.7773314714431763 | 0.7773314714431763 |
| pearson_corr | 0.3725650906562805 | 0.3725650906562805 | 0.3725650906562805 |
| jensen_shannon_divergence | 3.0076024532318115 | 3.0076024532318115 | 3.0076024532318115 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.792931318283081 | 0.792931318283081 | 0.792931318283081 |
| pearson_corr | 0.09795872867107391 | 0.09795872867107391 | 0.09795872867107391 |
| jensen_shannon_divergence | 3.7049810886383057 | 3.7049810886383057 | 3.7049810886383057 |


----------------------------



### 0.01, relu, sigmoid

In [34]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8967217803001404 | 0.8967217803001404 | 0.8967217803001404 |
| pearson_corr | 0.021258065477013588 | 0.021258065477013588 | 0.021258065477013588 |
| jensen_shannon_divergence | 6.083329677581787 | 6.083329677581787 | 6.083329677581787 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5430918335914612 | 0.5430918335914612 | 0.5430918335914612 |
| pearson_corr | 0.5132556557655334 | 0.5132556557655334 | 0.5132556557655334 |
| jensen_shannon_divergence | 1.488609790802002 | 1.488609790802002 | 1.488609790802002 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8919869065284729 | 0.8919869065284729 | 0.8919869065284729 |
| pearson_corr | 0.03479147329926491 | 0.03479147329926491 | 0.03479147329926491 |
| jensen_shannon_divergence | 5.35507345199585 | 5.35507345199585 | 5.35507345199585 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.767339825630188 | 0.767339825630188 | 0.767339825630188 |
| pearson_corr | 0.3749082088470459 | 0.3749082088470459 | 0.3749082088470459 |
| jensen_shannon_divergence | 2.922717571258545 | 2.922717571258545 | 2.922717571258545 |


----------------------------



### 0.01, relu, relu

In [35]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7219480872154236 | 0.7219480872154236 | 0.7219480872154236 |
| pearson_corr | 0.19677577912807465 | 0.19677577912807465 | 0.19677577912807465 |
| jensen_shannon_divergence | 2.6213958263397217 | 2.6213958263397217 | 2.6213958263397217 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5489581227302551 | 0.5489581227302551 | 0.5489581227302551 |
| pearson_corr | 0.5024368166923523 | 0.5024368166923523 | 0.5024368166923523 |
| jensen_shannon_divergence | 1.5034209489822388 | 1.5034209489822388 | 1.5034209489822388 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6110338568687439 | 0.6110338568687439 | 0.6110338568687439 |
| pearson_corr | 0.5436323285102844 | 0.5436323285102844 | 0.5436323285102844 |
| jensen_shannon_divergence | 1.814046025276184 | 1.814046025276184 | 1.814046025276184 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128, 64], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7673341631889343 | 0.7673341631889343 | 0.7673341631889343 |
| pearson_corr | 0.3749210238456726 | 0.3749210238456726 | 0.3749210238456726 |
| jensen_shannon_divergence | 2.922659158706665 | 2.922659158706665 | 2.922659158706665 |


----------------------------

